# Gemini Video Processing

## Imports

In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os

load_dotenv()

True

## Gemini Implementation

In [ ]:
# Setup Gemini
class GeminiVideoProcessor:
    """Gemini Video Processor"""

    def __init__(self, model_name="gemini-2.5-flash"):
        """
        Initialize Gemini with a pre-trained model

        Args:
            model_name: HuggingFace model name for Gemini
        """
        #from google.colab import userdata
        #api_key = userdata.get('GEMINI_API_KEY')

        api_key = os.getenv("GEMINI_API_KEY")
        print(f"Loading Gemini model: {model_name}")
        self.client = genai.Client(api_key=api_key)
        print("✅ Gemini loaded successfully!")
        self.model_name = model_name

    def _wait_file_active(self, file_name: str, timeout: int = 600, interval: float = 2.0):
        """Poll the uploaded file until it becomes ACTIVE."""
        start = time.time()
        last_state = None
        while True:
            f = self.client.files.get(name=file_name)
            last_state = getattr(f, "state", None)
            if last_state == "ACTIVE":
                return f
            if last_state in ("FAILED", "DELETED"):
                raise RuntimeError(f"File {file_name} ended in state {last_state}.")
            if time.time() - start > timeout:
                raise TimeoutError(f"Timed out waiting for file {file_name} to become ACTIVE (last state={last_state}).")
            time.sleep(interval)
    
    def upload_video(self, video_path: str, prompt: str) -> str:
        myfile = self.client.files.upload(
            file=video_path,
            config=
            mime_type="video/mp4",
            display_name=os.path.basename(video_path)
        )
        myfile = self._wait_file_active(myfile.name)

        response = self.client.models.generate_content(model=self.model_name, contents=[myfile, prompt])

        return response.text
    
    def pass_video_data(self, video_path: str, prompt: str) -> str:
        # Only for videos of size <20Mb
        video_bytes = open(video_path, 'rb').read()

        response = self.client.models.generate_content(
            model=self.model_name,
            contents=types.Content(
                parts=[
                    types.Part(
                        inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
                    ),
                    types.Part(text=prompt)
                ]
            )
        )

        return response.text
    
    def pass_youtube_link(self, youtube_url: str, prompt: str) -> str:
        response = self.client.models.generate_content(
            model=self.model_name,
            contents=types.Content(
                parts=[
                    types.Part(
                        file_data=types.FileData(file_uri=youtube_url)
                    ),
                    types.Part(text=prompt)
                ]
            )
        )

        return response.text

## Demo

In [73]:
gemini = GeminiVideoProcessor()

Loading Gemini model: gemini-2.5-flash
✅ Gemini loaded successfully!


In [74]:
for f in gemini.client.files.list():
    myfile = gemini.client.files.get(name=f.name)
    gemini.client.files.delete(name=myfile.name)
    

In [75]:
for f in gemini.client.files.list():
    print(f)

### Video Process with YouTube Link

In [20]:
gemini.pass_youtube_link("https://www.youtube.com/watch?v=W6ZHY0E4_Wg", "Summarize video in 3 sentences.")

'During a press conference, Donald Trump strongly criticized U.S. intelligence agencies for allowing "false and fake" information to be released to the public, comparing it to actions by Nazi Germany. He attacked media outlets like Buzzfeed and CNN for publishing these reports, citing a false claim about his lawyer, Michael Cohen, being in Prague as an example of fabricated information. He then emphatically refused to answer a question from a CNN reporter, pointing and repeating, "You are fake news."'

### Video Process with Local File >20 MB

In [77]:
gemini.upload_video("files/videos/lebron.mp4", "Summarize video in a sentence.")

TypeError: Files.upload() got an unexpected keyword argument 'mime_type'

### Video Process with Local File <20 MB

In [22]:
gemini.pass_video_data("files/videos/lebron.mp4", "Summarize video in a sentence.")

'The video features LeBron James discussing basketball players, then cuts to Stephen Curry and others entering a room, and concludes with LeBron, Stephen Curry, and another man toasting wine glasses.'